# MS Prediction Data Cleaning

In [1]:
import pandas as pd

import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

from dateutil import parser
import datetime

from sklearn.model_selection import train_test_split

## Diagnoses labels

Found in `seantis/diagnoses.csv` in the `disease` column.

In [2]:
labels = pd.read_csv(paths.DATA_PATH_SEANTIS/'diagnoses.csv')

In [3]:
display(labels.head())
print(labels.shape)

,diagnosis_date,diagnosis_reliability,disease,disease_onset_date,research_id
0,2011-10,confirmed,Progressive multifokale Leukencephalopathie (PML),2011-10,A0684D32-19C0-4538-AFDE-52959CCD8B63
1,2002-08,confirmed,relapsing_remitting_multiple_sclerosis,2002-06,A0684D32-19C0-4538-AFDE-52959CCD8B63
2,2011-11,confirmed,St.n. symptomatischer Epilepsie mit einfach fo...,2011-11,A0684D32-19C0-4538-AFDE-52959CCD8B63
3,2008-10,confirmed,relapsing_remitting_multiple_sclerosis,2003,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
4,2000,NaN,relapsing_remitting_multiple_sclerosis,2000,EF6AFC6C-BEB1-43B9-A5F4-79F321FF100C


(387, 5)


In [4]:
print("Label distribution:")
print(labels['disease'].value_counts())
print("None: ", labels['disease'].isna().sum(), "\n")
print("Unique patient IDs: ", len(labels['research_id'].unique()))
print("Duplicate patient IDs: ", len(labels['research_id']) - len(labels['research_id'].unique()))

Label distribution:
disease
relapsing_remitting_multiple_sclerosis                                              284
clinically_isolated_syndrome                                                         45
secondary_progressive_multiple_sclerosis                                             27
primary_progressive_multiple_sclerosis                                               18
Schubförmige Multiple Sklerose                                                        3
Multiple Sklerose a.e. primär progredient                                             2
Progressive multifokale Leukencephalopathie (PML)                                     1
St.n. symptomatischer Epilepsie mit einfach fokalen und komplex fokalen Anfällen      1
Schubförmig remittierende Multiple Sklerose (RRMS)                                    1
Multiple Sklerose mit am ehesten progredientem Verlauf                                1
Schubförmig remittierende Multiple Sklerose                                           1
Mult

In [5]:
print("Diagnosis reliability:")
print(labels['diagnosis_reliability'].value_counts())

Diagnosis reliability:
diagnosis_reliability
confirmed      267
status_post     51
suspected       12
Name: count, dtype: int64


In [6]:
# Print missing dates
print("Missing dates:")
print(labels["diagnosis_date"].isna().sum())
print("\n")

# Date column formats
print("Date column formats:")
print(labels["diagnosis_date"].apply(lambda x: len(x)).value_counts())
print("\n")

# Print a 10 character date
print("Example date:")
print(labels["diagnosis_date"][labels["diagnosis_date"].apply(lambda x: len(x)) == 10].iloc[1])

Missing dates:
0


Date column formats:
diagnosis_date
7     270
4      67
10     50
Name: count, dtype: int64


Example date:
2020-10-30


In [7]:
# Guessing the date format from the length and example. Seems to follow english format with YYYY-MM-DD.
date_map = {
    7: "YYYY-MM",
    10: "YYYY-MM-DD",
    4: "YYYY",
}
print(labels["diagnosis_date"].apply(lambda x: len(x))
      .value_counts()
      .reset_index()
      .replace({"diagnosis_date": date_map})
      .rename(columns={"diagnosis_date": "format"}))

       format  count
0     YYYY-MM    270
1        YYYY     67
2  YYYY-MM-DD     50


In [8]:
# Print the entries for the patients with multiple labels
print("Examples Patients with multiple labels:")
display((labels[labels["research_id"].duplicated(keep=False)].sort_values(by=["research_id", "disease_onset_date"])))

Examples Patients with multiple labels:


,diagnosis_date,diagnosis_reliability,disease,disease_onset_date,research_id
114,2013-11,confirmed,relapsing_remitting_multiple_sclerosis,2012-12,05157228-A1C5-4335-B990-AF72B312FF1A
115,2013-04,status_post,clinically_isolated_syndrome,2012-12,05157228-A1C5-4335-B990-AF72B312FF1A
163,2015-02,confirmed,relapsing_remitting_multiple_sclerosis,2014-09,08F1FA27-6213-47AF-B967-DC44CCCE9613
164,2014-10,status_post,clinically_isolated_syndrome,2014-09-28,08F1FA27-6213-47AF-B967-DC44CCCE9613
63,2009-11,confirmed,relapsing_remitting_multiple_sclerosis,2009-01,146E4473-897F-402D-9F33-7943658FB47D
...,...,...,...,...,...
304,1997-10,confirmed,secondary_progressive_multiple_sclerosis,1992,F4D0C12B-8BEC-4B0E-BB80-44909D9A8507
188,2015-07,status_post,clinically_isolated_syndrome,2015-06,F5C813F0-1281-4C7E-87E1-78BA8AC3738A
189,2016-01,confirmed,relapsing_remitting_multiple_sclerosis,2015-06,F5C813F0-1281-4C7E-87E1-78BA8AC3738A
176,2017-03,confirmed,relapsing_remitting_multiple_sclerosis,2015-04,F713ABCC-6E83-45E5-B660-E68CA43024CF


### Problems

- Mix of german and english labels.
- Irrelevant labels
- Class imbalance
- There could be multiple diagnoses per patient
- The date column contains multiple formats, YYYY-MM, YYYY, YYYY-MM-DD
- Some diagnoses are not confirmed, could be detrimental to model training. In old approach they excluded these.

### Conclusion

- In the old project they mention, that only english labels are reliable. And they only use confirmed diagnoses for training. Manually check if the german labels are accurate.
- Exclude irrelevant labels (like clinically_isolated_syndrome). Keep relevant labels (relapsing_remitting_multiple_sclerosis (RRMS), secondary_progressive_multiple_sclerosis (SPMS), primary_progressive_multiple_sclerosis (PPMS))
- Do dataset-splits stratified
- Match with text from `kisim_diagnosis` based on research_id, check again how much remains.
- Multiple diagnoses per patient are reasonable, but problem is automatically solved by excluding non-confirmed diagnoses as all of the RRMS diagnoses were made status_post which shouldn't be info in the text.
- Clean date column by using YYYY-MM-DD format, mapping YYYY to the YYYY-01-01 and YYYY-MM to YYYY-MM-01
- Exclude non-confirmed diagnoses like in old approach.

## Diagnosis text

The diagnosis text is in `kisim_diagnoses.csv`

In [9]:
text = pd.read_csv(paths.DATA_PATH_SEANTIS/'kisim_diagnoses.csv')

In [10]:
text.head()

,data_provider_code,diagnosis_code,diagnosis_code_system,diagnosis_code_system_version,diagnosis_date,diagnosis_encoded,diagnosis_id,diagnosis_label,diagnosis_ranking,diagnosis_role,LastUpdateDateTime,research_id
0,CHE-108.904.325,NaN,NaN,NaN,NaN,no,7024338|1,Kutane Karzinogenese \r\nSpinozelläre Karzinom...,primary,discharge,2021-01-19 08:34:58.0800000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
1,CHE-108.904.325,NaN,NaN,NaN,NaN,no,5255555|1,Aktuell kompensierte Harnblasensituation bei s...,primary,discharge,2016-10-07 14:06:05.2630000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
2,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6435406|1,Aktuell kompensierte Harnblasensituation bei s...,primary,discharge,2020-04-28 06:27:53.2300000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
3,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6352273|5,"Schubförmige Multiple Sklerose\r\nES 03, ED 10...",secondary,discharge,2019-07-24 11:21:01.4300000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
4,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6352273|4,Aktuell kompensierte Harnblasensituation bei s...,secondary,discharge,2019-07-24 11:21:01.4300000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1


In [11]:
print("Description of text dataset: \n")
print(text.describe().iloc[:2], "\n")

print("Shape of text dataset: \n")
print(text.shape, "\n")

print("None values: \n")
print(text.isna().sum(), "\n")

Description of text dataset: 

       data_provider_code diagnosis_code diagnosis_code_system  \
count                7238           2101                  2101   
unique                  1            514                     1   

       diagnosis_code_system_version diagnosis_date diagnosis_encoded  \
count                           1808           2067              7238   
unique                             6            415                 2   

       diagnosis_id diagnosis_label diagnosis_ranking diagnosis_role  \
count          7238            7236              7238           7238   
unique         7238            3498                 2              1   

       LastUpdateDateTime research_id  
count                7238        7238  
unique               1505         275   

Shape of text dataset: 

(7238, 12) 

None values: 

data_provider_code                  0
diagnosis_code                   5137
diagnosis_code_system            5137
diagnosis_code_system_version    5430
diagno

In [12]:
# Unique patient IDs
print("Unique patient IDs: ", len(text['research_id'].unique()))

# Is there at least one date per research_id?
print("There is at least one date per rid: ", text.groupby("research_id")["diagnosis_date"].apply(lambda x: x.notna().sum().all()))

Unique patient IDs:  275
There is at least one date per rid:  research_id
016B6D16-2BBA-4C05-A8E4-30F761C95813     True
08301EF0-1057-41AB-9911-772AE295E282    False
086D33C8-DDD3-4DE2-BFE0-A945076A2335    False
088B7604-2F11-459A-8C8C-8E1A19943333     True
08F1FA27-6213-47AF-B967-DC44CCCE9613     True
                                        ...  
F85F7220-F7BB-40EC-B2D9-A32EF15FFB86    False
F876F155-CC5E-4512-AED1-2991D22B5DE9    False
F87876FA-3068-4A93-AE71-6071CBE5F1F2     True
F99EA55D-F709-47AD-A9C4-69764B3FEFF0     True
FD303233-49CC-47B2-A5B4-5A50EA63327B     True
Name: diagnosis_date, Length: 275, dtype: bool


In [13]:
# Date column formats
print("Date column formats:")
print(text["diagnosis_date"].dropna().apply(lambda x: len(x)).value_counts())

Date column formats:
diagnosis_date
27    2067
Name: count, dtype: int64


In [14]:
print("Example date:")
print(text["diagnosis_date"].dropna().iloc[0])

Example date:
2018-12-06 18:53:00.0000000


### Problems
- There are multiple text entries per rid (diagnosis_label column). This is because doctors mostly just appended to existing reports, meaning the longest one should be the newest, or fullest report. 
- The LastUpdateDate is not useful for the dates, as different rows, that correspond to the same entries, share this date
- The diagnosis_date column has a lot of missing values. Because of missing values matching is hard to match using date.
- The date column is formated in the YYYY-MM-DD hh:mm:ss.ms format, could reformat

### Conclusions
- Multiple entries per rid, I will use the longest. This was already done for the old project. The entries are stored in /midatams/preprocessed_nlp/midata-text-extraction/data/diagnoses/diags_seantis_kisim_longest.json
- Date column might not be used for analysis but might prove useful for time-series follow-up. After join, this column should be cleaned.

## Joining Datasets

We can join the datasets based on the rid. For rids with multiple entries we will use the date column to match them exactly. 

In [15]:
# Overlap of rid between labels and text
print("Unique patient IDs in labels: ", len(labels['research_id'].unique()))
print("Unique patient IDs in text: ", len(text['research_id'].unique()))
print("Overlap of research_id between labels and text: ", len(set(labels["research_id"]).intersection(set(text["research_id"]))))

# Double labels
# Join labels and text on rid
double_labeled = labels["research_id"][labels["research_id"].duplicated(keep="first")]
print("Number of double labeled patients in both datasets: ", len(set(double_labeled).intersection(set(text["research_id"]))))

Unique patient IDs in labels:  333
Unique patient IDs in text:  275
Overlap of research_id between labels and text:  257
Number of double labeled patients in both datasets:  36


### Processing steps
Steps: 
1. Merge, rename, drop non-confirmed diagnoses and irrelevant columns
2. German labels could be unreliable. Check manually, then relabel
3. Drop all entries that are not one of RRMS, PPMS or SPMS. 
4. primary progressive and secondary progressive have low counts. Very important that text matches these. Check manually.
5. Try to mine more SPMS or PPMS by checking entries that might have been wrongly labelled as RRMS and remap them.

#### 1. Merge, rename, drop

In [16]:
# Loading data. Text data is loaded from json file using the same data as in old approach
import json
labels = pd.read_csv(paths.DATA_PATH_SEANTIS/'diagnoses.csv').rename(columns={'research_id': 'rid'})
text = json.load(open(paths.DATA_PATH_PREPROCESSED/'midatams/diags_seantis_kisim_longest.json', 'r'))
text = {k.split('_')[0]: ' '.join(v) for k, v in text.items()} # File name was rid_something_date
text = pd.DataFrame.from_dict(text, orient='index').reset_index().rename(columns={0: 'text', 'index': 'rid'})

# Merge on rid, drop irrelevant columns, rename columns
df = pd.merge(labels, text, on='rid', how='inner')

# Only use confirmed diagnosis
df = df[df["diagnosis_reliability"] == "confirmed"]

# Remove and rename columns
df = df[["rid", "diagnosis_date", "disease", "text"]].rename(columns={"diagnosis_date": "date", "disease": "labels"})

#### 2. German labels

In [17]:
# Check special cases where I rewrite the label
map_dict = {
    "Schubförmig remittierende Multiple Sklerose (RRMS)": "relapsing_remitting_multiple_sclerosis",
    "Schubförmig remittierende Multiple Sklerose": "relapsing_remitting_multiple_sclerosis",
    "Schubförmig remittierende Multiple Sklerose ": "relapsing_remitting_multiple_sclerosis",
    "Multiple Sklerose mit V.a. sekundär chronisch-progredienten Verlauf": "secondary_progressive_multiple_sclerosis",
    "Multiple Sklerose a.e. primär progredient": "primary_progressive_multiple_sclerosis",
    "Multiple Sklerose mit a.e. primär-progredientem Verlauf": "primary_progressive_multiple_sclerosis",
}

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df[df["labels"].isin(map_dict.keys())]

rid     date  \
75   FD303233-49CC-47B2-A5B4-5A50EA63327B  2011-11   
216  11A4453B-76B6-4558-8126-EEFEA6935C91  2015-03   
236  D5CD046C-1F79-487E-9B8A-EFE9E108DCC5  2011-11   
252  EE72BED8-0777-40E2-A541-B9C1D993E5ED  2015-07   
283  9FFAE238-BD0D-4231-B754-4EDC64A6FCCC     2015   

                                                      labels  \
75        Schubförmig remittierende Multiple Sklerose (RRMS)   
216                Multiple Sklerose a.e. primär progredient   
236                Multiple Sklerose a.e. primär progredient   
252             Schubförmig remittierende Multiple Sklerose    
283  Multiple Sklerose mit a.e. primär-progredientem Verlauf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [18]:
# It seems text entry of 252 does not contain enough information to be useful The rest of the entries are fine and can be remapped
df.drop(252, inplace=True)

#### 3. Remapping, drop non interesting labels

In [19]:
# Remap labels
df = df.replace(map_dict)

# Only use confirmed diagnosis and the relevant labels
labels_list = ["relapsing_remitting_multiple_sclerosis", "secondary_progressive_multiple_sclerosis", "primary_progressive_multiple_sclerosis"]
df = df[df["labels"].isin(labels_list)]

#### 4. SPMS and PPMS manual check

In [20]:
# Because mapping was done manually, check if label matches text for classes with low counts
display(df[df["labels"] == "secondary_progressive_multiple_sclerosis"])

rid     date  \
11   72C0291F-085F-4AED-9526-80DC13318958     2005   
38   CB96C8EB-8D90-4D3A-B9D1-F02F1AD11377     2011   
47   B886879A-5109-46FD-A2B0-9DCA2DA733F8  2004-04   
50   2A9F4832-B09D-470A-B05F-519854310DBB     1990   
133  55DADCF5-ABAF-43E8-B8FC-10E2966E8A32     2002   
210  C0784569-1E15-4FBE-A4B2-F9473975D199     2004   
211  AF834D8D-F7DB-4B22-BB01-29F10EE6A828     2015   
213  E81AD81E-7504-4FFE-AA11-77E288E77CEC     2004   
218  39D432B0-902B-49D9-B727-12EDC053B09E  2015-11   
220  8BE4E431-291A-4722-B78F-3CD446550505     2003   
254  786DE63D-D0A9-4380-9BD6-3E5EF553376A     2006   
263  CACC4AEC-E2E4-429E-8697-5FC53727E31A  2010-02   
291  763EC115-4945-4536-A788-A1CC7B8A6A79     1999   

                                       labels  \
11   secondary_progressive_multiple_sclerosis   
38   secondary_progressive_multiple_sclerosis   
47   secondary_progressive_multiple_sclerosis   
50   secondary_progressive_multiple_sclerosis   
133  secondary_progressive_multiple_sclerosis   
210  secondary_progressive_multiple_sclerosis   
211  secondary_progressive_multiple_sclerosis   
213  secondary_progressive_multiple_sclerosis   
218  secondary_progressive_multiple_sclerosis   
220  secondary_progressive_multiple_sclerosis   
254  secondary_progressive_multiple_sclerosis   
263  secondary_progressive_multiple_sclerosis   
291  secondary_progressive_multiple_sclerosis   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [21]:
# For entries (47 (RRMS), 210 (unclear), 211 (RRMS), 218 (unclear)) text is not consistent with label. 
# I will remap entries 47 and 211 to RRMS and drop the other two entries.
df.drop([210, 218], inplace=True)
df.loc[df.index == 47, "labels"] = "relapsing_remitting_multiple_sclerosis"
df.loc[df.index == 211, "labels"] = "relapsing_remitting_multiple_sclerosis"

In [22]:
# Check primary_progressive_multiple_sclerosis
display(df[df["labels"] == "primary_progressive_multiple_sclerosis"])

rid        date  \
17   0A7AF4C2-ADD6-4B72-9869-4D190928F8C3        2012   
63   B9DF209B-BC2B-4B43-9985-9E6D245D67FF        2009   
95   5CA80E8A-71BC-48C0-B652-CDEBEF93AC28        1996   
138  B0B07B18-0211-4467-8489-581AC761D297        2012   
205  9B962F57-AF41-42EE-B7B5-A99BFCDBEE51     2018-10   
216  11A4453B-76B6-4558-8126-EEFEA6935C91     2015-03   
232  DA54B1C1-5867-4C53-9AAE-1B1C224729DE        1994   
236  D5CD046C-1F79-487E-9B8A-EFE9E108DCC5     2011-11   
259  745DA433-83BC-48B0-889D-9578F7F7C3D8     2017-08   
260  D32F0EA9-537A-49FA-805B-D6A02A8BF7E7     2017-08   
277  7C694045-8AFF-46EC-A410-D904065A52FC     2018-07   
283  9FFAE238-BD0D-4231-B754-4EDC64A6FCCC        2015   
293  9AFE6951-3D0F-4B6E-B989-124481C73826  2016-06-01   

                                     labels  \
17   primary_progressive_multiple_sclerosis   
63   primary_progressive_multiple_sclerosis   
95   primary_progressive_multiple_sclerosis   
138  primary_progressive_multiple_sclerosis   
205  primary_progressive_multiple_sclerosis   
216  primary_progressive_multiple_sclerosis   
232  primary_progressive_multiple_sclerosis   
236  primary_progressive_multiple_sclerosis   
259  primary_progressive_multiple_sclerosis   
260  primary_progressive_multiple_sclerosis   
277  primary_progressive_multiple_sclerosis   
283  primary_progressive_multiple_sclerosis   
293  primary_progressive_multiple_sclerosis   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [23]:
# Everything seems fine here

#### 5. Manual Mining of SPMS and PPMS

In [24]:
# Find entries that contain "SPMS" or sekundär and have label relapsing_remitting_multiple_sclerosis
df[(df["labels"] == "relapsing_remitting_multiple_sclerosis") & (df["text"].str.contains("SPMS|sekundär"))]

rid     date  \
24   0FA0E814-EFF9-4888-8410-C27659833E25  2004-08   
135  88F16735-6B66-448A-B88E-75DFBA12A6A2     2003   
160  328AFD0B-0ADD-4DA8-B4FB-4CA08D63781D  2012-05   
214  189A2297-8248-470D-8378-66B37264856C     1995   
292  A3141CEA-1444-4DDC-955F-12149A7BBFB9  2020-07   

                                     labels  \
24   relapsing_remitting_multiple_sclerosis   
135  relapsing_remitting_multiple_sclerosis   
160  relapsing_remitting_multiple_sclerosis   
214  relapsing_remitting_multiple_sclerosis   
292  relapsing_remitting_multiple_sclerosis   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

There are entries where a RRMS is diagnosed, but SPMS is suspected. Keep in mind when looking at results. I won't remap.

In [25]:
# Find entries that contain "PPMS" or primär and have label relapsing_remitting_multiple_sclerosis
df[(df["labels"] == "relapsing_remitting_multiple_sclerosis") & (df["text"].str.contains("PPMS|primär"))]

rid        date  \
14   54FBE2C1-4F4A-4E43-AD9B-CF5774675F73        2004   
34   6E251439-B21E-4826-A27E-251B5DFB8024     2017-07   
160  328AFD0B-0ADD-4DA8-B4FB-4CA08D63781D     2012-05   
246  E63331C8-F771-4020-BFEA-20CF57B1B707  2016-06-24   

                                     labels  \
14   relapsing_remitting_multiple_sclerosis   
34   relapsing_remitting_multiple_sclerosis   
160  relapsing_remitting_multiple_sclerosis   
246  relapsing_remitting_multiple_sclerosis   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

#### 6. Date reformat

In [26]:
# Reformat date column
df["date"] = df["date"].apply(lambda x: parser.parse(x, default=datetime.datetime(2023, 1, 1)).strftime("%Y-%m-%d"))

#### 7. Summary

In [27]:
# Summary of cleaned dataset
print("Summary of cleaned dataset: \n")
print(df.info(), "\n")
print("shape: ", df.shape, "\n")
print("label distribution: \n", df["labels"].value_counts(), "\n")
print("missing: \n", df.isna().sum(), "\n")

Summary of cleaned dataset: 

<class 'pandas.core.frame.DataFrame'>
Index: 196 entries, 1 to 293
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rid     196 non-null    object
 1   date    196 non-null    object
 2   labels  196 non-null    object
 3   text    196 non-null    object
dtypes: object(4)
memory usage: 7.7+ KB
None 

shape:  (196, 4) 

label distribution: 
 labels
relapsing_remitting_multiple_sclerosis      174
primary_progressive_multiple_sclerosis       13
secondary_progressive_multiple_sclerosis      9
Name: count, dtype: int64 

missing: 
 rid       0
date      0
labels    0
text      0
dtype: int64 



In [28]:
# Train test split
train, test = train_test_split(df, test_size=0.3, random_state=42, stratify=df["labels"])
train, val = train_test_split(train, test_size=0.1, random_state=42, stratify=train["labels"])

# Save to csv
train.to_csv(paths.DATA_PATH_PREPROCESSED/'ms-diag/ms-diag_clean_train.csv', index=False)
val.to_csv(paths.DATA_PATH_PREPROCESSED/'ms-diag/ms-diag_clean_val.csv', index=False)
test.to_csv(paths.DATA_PATH_PREPROCESSED/'ms-diag/ms-diag_clean_test.csv', index=False)

In [29]:
print("Train label distribution: \n", train["labels"].value_counts(), "\n")
print("Val label distribution: \n", val["labels"].value_counts(), "\n")
print("Test label distribution: \n", test["labels"].value_counts(), "\n")

Train label distribution: 
 labels
relapsing_remitting_multiple_sclerosis      110
primary_progressive_multiple_sclerosis        8
secondary_progressive_multiple_sclerosis      5
Name: count, dtype: int64 

Val label distribution: 
 labels
relapsing_remitting_multiple_sclerosis      12
secondary_progressive_multiple_sclerosis     1
primary_progressive_multiple_sclerosis       1
Name: count, dtype: int64 

Test label distribution: 
 labels
relapsing_remitting_multiple_sclerosis      52
primary_progressive_multiple_sclerosis       4
secondary_progressive_multiple_sclerosis     3
Name: count, dtype: int64 



# Old Project Preprocessing
To get comparability to old project I will use parts of their data preprocessing. Specifically I use the seantis_kisim.csv that they have prepared. Their approach:

1. Extract the longest diagnosis per rid (most lines) from the csv and if the rid had a manually line labelled text, they used this instead.
2. Results in dataset consisting of text lines per row with a label for the line.

Further processing:
3. I can use this to create a df with joined text per rid and determined what content will be in the text. Later I can filter useful examples based on this.
4. If a content entry is False, we know that there is no such information in the text. This is useful to estimate the accuracy of a model to correctly determine the lack of information. If the entry is NaN we don't know if the information is present.
5. As there are very few entries for SPMS and PPMS I will manually check their lines for dm content. If I make any changes to line labellings I will report this in the "line_label_origin" column as "manual", while original labels are stored as "original". Later I also add a third option for "classifier1" where a Model automatically labels the lines.
6. As there are very few entries for SPMS and PPMS I will manually check their labels for correctness. If I change any of their labels I will remark this in the "label_origin" column as "manual", while original labels will be stored as "original". Later I also add a third option for "classifier2" where a Model automatically labels the text.

In [115]:
import pandas as pd
import numpy as np

In [116]:
df = pd.read_csv(os.path.join(paths.DATA_PATH_PREPROCESSED,"line_labelling", "seantis_kisim.csv"))

In [117]:
# Check if rid text contains a certain content
contains_variable_dict = {line_class: f"contains_{line_class}" for line_class in df["class"].unique()}

# Create columns
df_clean = pd.DataFrame(columns=["rid", "text"] + list(contains_variable_dict.values()) + ["line_label_origin"])
line_labelled_reports = 0

# Fill df_clean
for rid, rid_data in df.groupby("research_id"):

    # Concatenate text lines
    text = rid_data["text"].dropna().str.cat(sep="\n")

     # If all line classes are nan fill with nan
    if rid_data["class"].isna().all():
        _df = pd.DataFrame({"rid": rid, "text": text, **{k: np.nan for k in contains_variable_dict.values()}, "line_label_origin": np.nan}, index=[0])
        df_clean = pd.concat([df_clean, _df], ignore_index=True)
    
    # If value is contained fill with True, else False
    else:
        text_contains = rid_data["class"].unique()
        text_contains_dict = {v: False for k, v in contains_variable_dict.items()}
        text_contains_dict.update({contains_variable_dict[v]: True for v in text_contains})
        _df = pd.DataFrame({"rid": rid, "text": text, **text_contains_dict, "line_label_origin": "original"}, index=[0])
        df_clean = pd.concat([df_clean, _df], ignore_index=True)
        line_labelled_reports += 1

In [118]:
print("Number of line labelled reports: ", line_labelled_reports)
print("Number of reports without line labels: ", len(df_clean) - line_labelled_reports)
print("Number of reports in df_clean: ", len(df_clean))

Number of line labelled reports:  74
Number of reports without line labels:  225
Number of reports in df_clean:  299


In [119]:
# Loading data. Text data is loaded from json file using the same data as in old approach
labels = pd.read_csv(paths.DATA_PATH_SEANTIS/'diagnoses.csv').rename(columns={'research_id': 'rid'}).drop(columns=["diagnosis_date", "disease_onset_date"])

# Add label_origin and fill with "original"
labels["label_origin"] = "original"

# Merge on rid, drop irrelevant columns, rename columns
df_merged = pd.merge(labels, df_clean, on='rid', how='inner')

# Only use confirmed diagnosis
df_merged = df_merged[df_merged["diagnosis_reliability"] == "confirmed"].drop(columns=["diagnosis_reliability"])

# Remove and rename columns
df_merged = df_merged.rename(columns={"disease": "labels"})

# Check special cases where I rewrite the label
map_dict = {
    "Schubförmig remittierende Multiple Sklerose (RRMS)": "relapsing_remitting_multiple_sclerosis",
    "Schubförmig remittierende Multiple Sklerose": "relapsing_remitting_multiple_sclerosis",
    "Schubförmig remittierende Multiple Sklerose ": "relapsing_remitting_multiple_sclerosis",
    "Multiple Sklerose mit V.a. sekundär chronisch-progredienten Verlauf": "secondary_progressive_multiple_sclerosis",
    "Multiple Sklerose a.e. primär progredient": "primary_progressive_multiple_sclerosis",
    "Multiple Sklerose mit a.e. primär-progredientem Verlauf": "primary_progressive_multiple_sclerosis",
}

# Remap labels
df_merged = df_merged.replace(map_dict)

# Only use confirmed diagnosis and the relevant labels
labels_list = ["relapsing_remitting_multiple_sclerosis", "secondary_progressive_multiple_sclerosis", "primary_progressive_multiple_sclerosis"]
df_merged = df_merged[df_merged["labels"].isin(labels_list)]


In [120]:
df_merged[df_merged.contains_dm == True].labels.value_counts()

labels
relapsing_remitting_multiple_sclerosis    28
primary_progressive_multiple_sclerosis     3
Name: count, dtype: int64

In [121]:
df_merged[df_merged.labels == "secondary_progressive_multiple_sclerosis"].isna().sum()

labels                0
rid                   0
label_origin          0
text                  0
contains_nan         13
contains_dm          13
contains_mr          13
contains_sym         13
contains_medms       13
contains_head        13
contains_his         13
contains_labr        13
contains_tr          13
contains_do          13
contains_so          13
contains_cu          13
contains_to          13
contains_medo        13
contains_labo        13
contains_unk         13
line_label_origin    13
dtype: int64

For PPMS and SPMS there are not enough line labels. I will check myself for ms and set to true if there is MS in report and False elsewhere.

In [122]:
df_merged[df_merged.labels == "secondary_progressive_multiple_sclerosis"]

labels  \
11   secondary_progressive_multiple_sclerosis   
38   secondary_progressive_multiple_sclerosis   
47   secondary_progressive_multiple_sclerosis   
50   secondary_progressive_multiple_sclerosis   
133  secondary_progressive_multiple_sclerosis   
210  secondary_progressive_multiple_sclerosis   
211  secondary_progressive_multiple_sclerosis   
213  secondary_progressive_multiple_sclerosis   
218  secondary_progressive_multiple_sclerosis   
220  secondary_progressive_multiple_sclerosis   
254  secondary_progressive_multiple_sclerosis   
263  secondary_progressive_multiple_sclerosis   
291  secondary_progressive_multiple_sclerosis   

                                      rid label_origin  \
11   72C0291F-085F-4AED-9526-80DC13318958     original   
38   CB96C8EB-8D90-4D3A-B9D1-F02F1AD11377     original   
47   B886879A-5109-46FD-A2B0-9DCA2DA733F8     original   
50   2A9F4832-B09D-470A-B05F-519854310DBB     original   
133  55DADCF5-ABAF-43E8-B8FC-10E2966E8A32     original   
210  C0784569-1E15-4FBE-A4B2-F9473975D199     original   
211  AF834D8D-F7DB-4B22-BB01-29F10EE6A828     original   
213  E81AD81E-7504-4FFE-AA11-77E288E77CEC     original   
218  39D432B0-902B-49D9-B727-12EDC053B09E     original   
220  8BE4E431-291A-4722-B78F-3CD446550505     original   
254  786DE63D-D0A9-4380-9BD6-3E5EF553376A     original   
263  CACC4AEC-E2E4-429E-8697-5FC53727E31A     original   
291  763EC115-4945-4536-A788-A1CC7B8A6A79     original   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [123]:
# From inspection it seems entries 210 and 218 do not contain dm, the rest does. Also entries 47 and 211 are relapsing_remitting_multiple_sclerosis and contain dm.
# Set contains_dm to True for all secondary_progressive_multiple_sclerosis entries
df_merged.loc[df_merged.labels == "secondary_progressive_multiple_sclerosis", "contains_dm"] = True
df_merged.loc[df_merged.labels == "secondary_progressive_multiple_sclerosis", "line_label_origin"] = "manual"
df_merged.loc[df_merged.index == 210, "contains_dm"] = False
df_merged.loc[df_merged.index == 218, "contains_dm"] = False

# Set label of entries 47, 50 and 211 to relapsing_remitting_multiple_sclerosis
df_merged.loc[df_merged.index == 47, "labels"] = "relapsing_remitting_multiple_sclerosis"
df_merged.loc[df_merged.index == 47, "label_origin"] = "manual"
df_merged.loc[df_merged.index == 211, "labels"] = "relapsing_remitting_multiple_sclerosis"
df_merged.loc[df_merged.index == 211, "label_origin"] = "manual"

In [89]:
# Check primary_progressive_multiple_sclerosis
df_merged[df_merged.labels == "primary_progressive_multiple_sclerosis"]

labels  \
17   primary_progressive_multiple_sclerosis   
63   primary_progressive_multiple_sclerosis   
95   primary_progressive_multiple_sclerosis   
138  primary_progressive_multiple_sclerosis   
205  primary_progressive_multiple_sclerosis   
216  primary_progressive_multiple_sclerosis   
232  primary_progressive_multiple_sclerosis   
236  primary_progressive_multiple_sclerosis   
259  primary_progressive_multiple_sclerosis   
260  primary_progressive_multiple_sclerosis   
277  primary_progressive_multiple_sclerosis   
283  primary_progressive_multiple_sclerosis   
293  primary_progressive_multiple_sclerosis   

                                      rid  \
17   0A7AF4C2-ADD6-4B72-9869-4D190928F8C3   
63   B9DF209B-BC2B-4B43-9985-9E6D245D67FF   
95   5CA80E8A-71BC-48C0-B652-CDEBEF93AC28   
138  B0B07B18-0211-4467-8489-581AC761D297   
205  9B962F57-AF41-42EE-B7B5-A99BFCDBEE51   
216  11A4453B-76B6-4558-8126-EEFEA6935C91   
232  DA54B1C1-5867-4C53-9AAE-1B1C224729DE   
236  D5CD046C-1F79-487E-9B8A-EFE9E108DCC5   
259  745DA433-83BC-48B0-889D-9578F7F7C3D8   
260  D32F0EA9-537A-49FA-805B-D6A02A8BF7E7   
277  7C694045-8AFF-46EC-A410-D904065A52FC   
283  9FFAE238-BD0D-4231-B754-4EDC64A6FCCC   
293  9AFE6951-3D0F-4B6E-B989-124481C73826   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [126]:
# Seems everything is fine with PPMS all contain dm
df_merged.loc[df_merged.labels == "primary_progressive_multiple_sclerosis", "contains_dm"] = True
df_merged.loc[df_merged.labels == "primary_progressive_multiple_sclerosis", "line_label_origin"] = "manual"

# Summary

In [136]:
# Label distribution
print("Label distribution all:")
print(df_merged.labels.value_counts(), "\n\n")

# Texts that contain diagnosis label
print("Label distribution contains_dm:")
print(df_merged[df_merged.contains_dm == True].labels.value_counts())

Label distribution all:
labels
relapsing_remitting_multiple_sclerosis      175
primary_progressive_multiple_sclerosis       13
secondary_progressive_multiple_sclerosis     11
Name: count, dtype: int64 


Label distribution contains_dm:
labels
relapsing_remitting_multiple_sclerosis      30
primary_progressive_multiple_sclerosis      13
secondary_progressive_multiple_sclerosis     9
Name: count, dtype: int64


In [137]:
# Save to csv
df_merged.to_csv(paths.DATA_PATH_PREPROCESSED/'ms-diag/ms-diag_line_annotated.csv', index=False)